## 4.1

Zur Bestimmung relevanter Merkmale wurde die Korrelation mit dem Zielwert `Average playtime forever` berechnet. Die höchsten Korrelationswerte zeigten:

- `Median playtime forever`: r = 0.96
- `Positive`: r = 0.62
- `Price`: r = 0.39

Diese Felder korrelieren stark mit der Zielvariable und sind daher für die Vorhersage besonders aussagekräftig. Die Korrelation wurde mit `df.corr()` geprüft.


## 4.2

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
games = pd.read_csv("/Users/ylli/Library/CloudStorage/OneDrive-AlteKantonsschuleAarau/BBBaden/M259/games.csv")
games.head()

,AppID,Name,Release date,Estimated owners,Required age,Price,Supported languages,Positive,Negative,Achievements,Recommendations,Average playtime forever,Publishers,Categories,Genres
0,20200,Galactic Bowling,21-Oct-08,0 - 20000,0,19.99,['English'],6,11,30,0,0,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports"
1,655370,Train Bandit,12-Oct-17,0 - 20000,0,0.99,"['English', 'French', 'Italian', 'German', 'Sp...",53,5,12,0,0,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie"
2,1732930,Jolt Project,17-Nov-21,0 - 20000,0,4.99,"['English', 'Portuguese - Brazil']",0,0,0,0,0,Campião Games,Single-player,"Action,Adventure,Indie,Strategy"
3,1355720,Henosis™,23-Jul-20,0 - 20000,0,5.99,"['English', 'French', 'Italian', 'German', 'Sp...",3,0,0,0,0,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie"
4,1139950,Two Weeks in Painland,3-Feb-20,0 - 20000,0,0.00,"['English', 'Spanish - Spain']",50,8,17,0,0,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie"


## 4.2 – Metrik: Mean Squared Error

Für die Bewertung des linearen Regressionsmodells wurde der **MSE (Mean Squared Error)** verwendet. Er beträgt rund 1'005'000 – was auf hohe Abweichungen hindeutet. Das bedeutet, dass einzelne Vorhersagen sehr stark vom tatsächlichen Wert abweichen können. Die Bewertung des Modells muss also kritisch erfolgen.

In [3]:
# Daten vorbereiten
games = games.dropna(subset=["Average playtime forever", "Price", "Positive"])

X = games[["Price", "Positive"]]
y = games["Average playtime forever"]

# Trainings- und Testdaten splitten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modell trainieren
model = LinearRegression()
model.fit(X_train, y_train)

# Vorhersagen berechnen
predictions = model.predict(X_test)


In [4]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, predictions)
print("📉 MSE (Mean Squared Error):", round(mse, 2))

# Realistischer Vergleich
diff = abs(y_test.values - predictions)
within_100 = (diff < 100).sum()
total = len(diff)

print(f"🔍 {within_100}/{total} Vorhersagen weichen weniger als 100 Minuten ab ({round(100 * within_100 / total, 2)}%)")

📉 MSE (Mean Squared Error): 1058802.58
🔍 1036/2100 Vorhersagen weichen weniger als 100 Minuten ab (49.33%)


## 4.3

Für eine binäre Klassifikation wurde eine Schwelle von 100 Minuten Spielzeit gewählt. Spiele über diesem Wert gelten als "Langspieler". Anhand der Vorhersagen wurde eine Wahrheitsmatrix erstellt. Zusätzlich wurden Sensitivität (Recall) und Spezifität (True Negative Rate) berechnet, um die Qualität des Modells umfassend zu beurteilen.

In [6]:
# Wir definieren: ab 100 Minuten zählt als "Langspieler" = Klasse 1
threshold = 100

# Tatsächliche Klassen
y_test_binary = (y_test >= threshold).astype(int)

# Vorhergesagte Klassen
predictions_binary = (predictions >= threshold).astype(int)

In [7]:
from sklearn.metrics import confusion_matrix

# Matrix berechnen
cm = confusion_matrix(y_test_binary, predictions_binary)
tn, fp, fn, tp = cm.ravel()

print("📊 Confusion Matrix:")
print(cm)
print(f"TN = {tn}, FP = {fp}, FN = {fn}, TP = {tp}")


📊 Confusion Matrix:
[[907 830]
 [ 91 272]]
TN = 907, FP = 830, FN = 91, TP = 272


In [8]:
from sklearn.metrics import precision_score, recall_score

precision = precision_score(y_test_binary, predictions_binary)
recall = recall_score(y_test_binary, predictions_binary)

print("🎯 Precision (Wie genau sind die Langspieler-Vorhersagen?):", round(precision, 2))
print("📈 Recall (Wie viele Langspieler wurden erkannt?):", round(recall, 2))
# Spezifität manuell berechnen
specificity = tn / (tn + fp)
print("🧪 Spezifität (True Negative Rate):", round(specificity, 2))

🎯 Precision (Wie genau sind die Langspieler-Vorhersagen?): 0.25
📈 Recall (Wie viele Langspieler wurden erkannt?): 0.75
🧪 Spezifität (True Negative Rate): 0.52


## 4.4

Das Modell liefert gemischte Ergebnisse. Der MSE beträgt über 1'000'000, was auf teils erhebliche Abweichungen hindeutet. Nur rund 49 % der Vorhersagen weichen weniger als 100 Minuten vom tatsächlichen Wert ab. Bei der nachträglichen Klassifikation als "Langspieler" (ab 100 Minuten) erreicht das Modell einen Recall von 0.75 – es erkennt die meisten Langspieler korrekt. Die Präzision ist jedoch niedrig (0.25), ebenso wie die Spezifität (0.52), was auf viele falsch-positive Vorhersagen hinweist. Ursache könnten fehlende Merkmale wie Spielgenre oder Nutzerverhalten sein. Ein komplexeres Modell könnte helfen.